In [5]:
#Basic imports
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'
from sklearn import metrics #used for labeling
from kmodes.kprototypes import KPrototypes

#plotly imports
import plotly as py
import chart_studio.plotly as cs
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [24]:
#df is our original DataFrame
X = pd.read_csv('scaled_dummy_reduced_df.csv', index_col=0)
X.head()

,number_of_trips_taken,qtemphigh,qtemplow,qprecipitation,qday_Friday,qday_Monday,qday_Saturday,qday_Sunday,qday_Thursday,qday_Tuesday,qday_Wednesday,qday1typical_Not at all typical,qday1typical_Not very typical,qday1typical_Somewhat typical,qday1typical_Very typical,qborough_home_Brooklyn,qborough_home_Manhattan,qborough_home_Queens,qborough_home_Staten Island,qborough_home_The Bronx,qsurveyzone_home_Inner Brooklyn,qsurveyzone_home_Inner Queens,qsurveyzone_home_Manhattan Core,qsurveyzone_home_Middle Queens,qsurveyzone_home_Northern Bronx,qsurveyzone_home_Northern Manhattan,qsurveyzone_home_Outer Brooklyn,qsurveyzone_home_Outer Queens,qsurveyzone_home_Southern Bronx,qsurveyzone_home_Staten Island,qtripdaytime_AM,qtripdaytime_NOON,qtripdaytime_PM,qborough_start_Brooklyn,qborough_start_Inner Brooklyn,qborough_start_Inner Queens,qborough_start_Manhattan,qborough_start_Manhattan Core,qborough_start_Middle Queens,qborough_start_Northern Bronx,qborough_start_Northern Manhattan,qborough_start_Outer Brooklyn,qborough_start_Outer Queens,qborough_start_Outside of NYC,qborough_start_Queens,qborough_start_Southern Bronx,qborough_start_Staten Island,qborough_start_The Bronx,surveyzone_start_Inner Brooklyn,surveyzone_start_Inner Queens,surveyzone_start_Manhattan Core,surveyzone_start_Middle Queens,surveyzone_start_NOT CODED,surveyzone_start_Northern Bronx,surveyzone_start_Northern Manhattan,surveyzone_start_Outer Brooklyn,surveyzone_start_Outer Queens,surveyzone_start_Southern Bronx,surveyzone_start_Staten Island,qday1tripend_Child's daycare facility or school,qday1tripend_Doctor's office or hospital,"qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",qday1tripend_Friend or family member's home,"qday1tripend_Grocery store or market (including deli, bodega, etc.)",qday1tripend_Home,qday1tripend_Other,qday1tripend_Outside of New York City,qday1tripend_Park/Recreational area/Gym,qday1tripend_Restaurant or bar,"qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripend_School,qday1tripend_Work,qborough_end_Brooklyn,qborough_end_Manhattan,qborough_end_Outside of NYC,qborough_end_Queens,qborough_end_Staten Island,qborough_end_The Bronx,surveyzone_end_Inner Brooklyn,surveyzone_end_Inner Queens,surveyzone_end_Manhattan Core,surveyzone_end_Middle Queens,surveyzone_end_Northern Bronx,surveyzone_end_Northern Manhattan,surveyzone_end_Outer Brooklyn,surveyzone_end_Outer Queens,surveyzone_end_Southern Bronx,surveyzone_end_Staten Island,qday1tripendcode_Airport,qday1tripendcode_Bus stop,qday1tripendcode_Child's daycare facility or school,qday1tripendcode_Commuter rail station,qday1tripendcode_Doctor's office or hospital,"qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",qday1tripendcode_Friend or family member's home,qday1tripendcode_Grand Central Station,"qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",qday1tripendcode_Home,qday1tripendcode_Other,qday1tripendcode_PATH Station,qday1tripendcode_Park and ride/parking lot,qday1tripendcode_Park/Recreational area/Gym,qday1tripendcode_Penn Station,qday1tripendcode_Restaurant or bar,"qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",qday1tripendcode_Road/tunnel/bridge,qday1tripendcode_School,qday1tripendcode_Work,qday1triplength_cat_0 to 5,qday1triplength_cat_11 to 15,qday1triplength_cat_16 to 20,qday1triplength_cat_180+,qday1triplength_cat_21 to 25,qday1triplength_cat_26 to 30,qday1triplength_cat_31 to 35,qday1triplength_cat_36 to 40,qday1triplength_cat_41 to 45,qday1triplength_cat_46 to 50,qday1triplength_cat_51 to 55,qday1triplength_cat_56 to 60,qday1triplength_cat_6 to 10,qday1triplength_cat_61 to 180,qdisability1_No,qdisability1_Yes,qdisability2_No,qdisability2_Yes,qdisability3_No,qdisability3_Yes,qdisability4_No,qdisability4_Yes,qdisability5_No,qdisability5_Yes,qdisability6_No,qdisability6_Yes,qdisability7_No,qdisability7_Yes,qdisability8_No,qdisability8_Yes,qdisability9_No,qdisability9_Yes,qage

In [25]:
numer = X[["number_of_trips_taken", "qtemphigh", "qtemplow", "qprecipitation"]]

In [26]:
categ = X[["qday_Friday","qday_Monday","qday_Saturday","qday_Sunday","qday_Thursday","qday_Tuesday",
           "qday_Wednesday","qday1typical_Not at all typical",
           "qday1typical_Not very typical","qday1typical_Somewhat typical",
           "qday1typical_Very typical","qborough_home_Brooklyn","qborough_home_Manhattan",
           "qborough_home_Queens","qborough_home_Staten Island","qborough_home_The Bronx",
           "qsurveyzone_home_Inner Brooklyn","qsurveyzone_home_Inner Queens",
           "qsurveyzone_home_Manhattan Core","qsurveyzone_home_Middle Queens",
           "qsurveyzone_home_Northern Bronx","qsurveyzone_home_Northern Manhattan",
           "qsurveyzone_home_Outer Brooklyn","qsurveyzone_home_Outer Queens",
           "qsurveyzone_home_Southern Bronx", "qsurveyzone_home_Staten Island",
           "qtripdaytime_AM","qtripdaytime_NOON","qtripdaytime_PM","qborough_start_Brooklyn",
           "qborough_start_Inner Brooklyn",
           "qborough_start_Inner Queens","qborough_start_Manhattan","qborough_start_Manhattan Core",
           "qborough_start_Middle Queens","qborough_start_Northern Bronx","qborough_start_Northern Manhattan",
           "qborough_start_Outer Brooklyn","qborough_start_Outer Queens","qborough_start_Outside of NYC","qborough_start_Queens",
           "qborough_start_Southern Bronx","qborough_start_Staten Island",
           "qborough_start_The Bronx","surveyzone_start_Inner Brooklyn","surveyzone_start_Inner Queens",
           "surveyzone_start_Manhattan Core","surveyzone_start_Middle Queens","surveyzone_start_NOT CODED","surveyzone_start_Northern Bronx",
           "surveyzone_start_Northern Manhattan","surveyzone_start_Outer Brooklyn","surveyzone_start_Outer Queens",
           "surveyzone_start_Southern Bronx","surveyzone_start_Staten Island","qday1tripend_Child's daycare facility or school",
           "qday1tripend_Doctor's office or hospital","qday1tripend_Entertainment event (i.e. sporting event, play, etc.)",
           "qday1tripend_Friend or family member's home","qday1tripend_Grocery store or market (including deli, bodega, etc.)","qday1tripend_Home",
           "qday1tripend_Other","qday1tripend_Outside of New York City","qday1tripend_Park/Recreational area/Gym","qday1tripend_Restaurant or bar",
           "qday1tripend_Retail store (e.g. clothing, electronic, hardware, etc.)",
           "qday1tripend_School","qday1tripend_Work","qborough_end_Brooklyn",
           "qborough_end_Manhattan","qborough_end_Outside of NYC","qborough_end_Queens",
           "qborough_end_Staten Island","qborough_end_The Bronx",
           "surveyzone_end_Inner Brooklyn","surveyzone_end_Inner Queens",
           "surveyzone_end_Manhattan Core","surveyzone_end_Middle Queens",
           "surveyzone_end_Northern Bronx","surveyzone_end_Northern Manhattan",
           "surveyzone_end_Outer Brooklyn","surveyzone_end_Outer Queens",
           "surveyzone_end_Southern Bronx","surveyzone_end_Staten Island","qday1tripendcode_Airport",
           "qday1tripendcode_Bus stop","qday1tripendcode_Child's daycare facility or school",
           "qday1tripendcode_Commuter rail station","qday1tripendcode_Doctor's office or hospital",
           "qday1tripendcode_Entertainment event (i.e. sporting event, play, etc.)",
           "qday1tripendcode_Friend or family member's home","qday1tripendcode_Grand Central Station",
           "qday1tripendcode_Grocery store or market (including deli, bodega, etc.)",
           "qday1tripendcode_Home","qday1tripendcode_Other","qday1tripendcode_PATH Station","qday1tripendcode_Park and ride/parking lot",
           "qday1tripendcode_Park/Recreational area/Gym","qday1tripendcode_Penn Station",
           "qday1tripendcode_Restaurant or bar","qday1tripendcode_Retail store (e.g. clothing, electronic, hardware, etc.)",
           "qday1tripendcode_Road/tunnel/bridge","qday1tripendcode_School","qday1tripendcode_Work",
           "qday1triplength_cat_0 to 5","qday1triplength_cat_11 to 15","qday1triplength_cat_16 to 20",
           "qday1triplength_cat_180+","qday1triplength_cat_21 to 25","qday1triplength_cat_26 to 30",
           "qday1triplength_cat_31 to 35","qday1triplength_cat_36 to 40","qday1triplength_cat_41 to 45",
           "qday1triplength_cat_46 to 50","qday1triplength_cat_51 to 55","qday1triplength_cat_56 to 60",
           "qday1triplength_cat_6 to 10","qday1triplength_cat_61 to 180","qdisability1_No","qdisability1_Yes","qdisability2_No",
           "qdisability2_Yes","qdisability3_No","qdisability3_Yes","qdisability4_No","qdisability4_Yes",
           "qdisability5_No","qdisability5_Yes","qdisability6_No","qdisability6_Yes","qdisability7_No",
           "qdisability7_Yes","qdisability8_No","qdisability8_Yes","qdisability9_No","qdisability9_Yes","qagecode_18-24","qagecode_25-34",
           "qagecode_35-44","qagecode_45-54","qagecode_55-64","qagecode_65 or older",
           "qlicense_No","qlicense_Yes","qcaraccess_I do not have access to a car",
           "qcaraccess_I do not personally own or lease a car, but I have access to a car belonging to a member of my household",
           "qcaraccess_I personally own or lease a car","qcaraccess_Other","qwelfare1_No",
           "qwelfare1_Yes","qwelfare2_No","qwelfare2_Yes","qwelfare3_No","qwelfare3_Yes","qwelfare4_No",
           "qwelfare4_Yes","qwelfare5_No","qwelfare5_Yes","qcitibike_No","qcitibike_Yes","qday1triptravelcode_sp_Car Service",
           "qday1triptravelcode_sp_Carpool","qday1triptravelcode_sp_Carshare","qday1triptravelcode_sp_Citi Bike",
           "qday1triptravelcode_sp_Community van/dollar van","qday1triptravelcode_sp_Commuter rail",
           "qday1triptravelcode_sp_Electric bicycle","qday1triptravelcode_sp_Express bus","qday1triptravelcode_sp_Green taxi",
           "qday1triptravelcode_sp_Local bus","qday1triptravelcode_sp_Motorcycle","qday1triptravelcode_sp_Other",
           "qday1triptravelcode_sp_Other ferry","qday1triptravelcode_sp_PATH train","qday1triptravelcode_sp_Paratransit/ Access-A-Ride",
           "qday1triptravelcode_sp_Personal bicycle","qday1triptravelcode_sp_Personal car",
           "qday1triptravelcode_sp_Ride-hail service such as Uber or Lyft","qday1triptravelcode_sp_Select bus service",
           "qday1triptravelcode_sp_Shared-ride service such a Uber Pool, Via, or Lyft Line",
           "qday1triptravelcode_sp_Staten Island ferry","qday1triptravelcode_sp_Subway","qday1triptravelcode_sp_Walk",
           "qday1triptravelcode_sp_Yellow taxi","qpurposerecode_Accompanying other traveler",
           "qpurposerecode_Business","qpurposerecode_Commute to/from work","qpurposerecode_Dining",
           "qpurposerecode_Medical visit (doctor's office)","qpurposerecode_Other","qpurposerecode_Personal errands",
           "qpurposerecode_School","qpurposerecode_Shopping","qpurposerecode_Social/recreation"]]


In [48]:
categ = categ.astype('category')

In [49]:
categ.dtypes

qday_Friday                                                                                                            category
qday_Monday                                                                                                            category
qday_Saturday                                                                                                          category
qday_Sunday                                                                                                            category
qday_Thursday                                                                                                          category
qday_Tuesday                                                                                                           category
qday_Wednesday                                                                                                         category
qday1typical_Not at all typical                                                                         

In [51]:
categoricals_indicies = []
for categ in categ:
        categoricals_indicies.append(categ.index(categ))

In [52]:
X.shape

(7851, 198)

In [53]:
#Initialize model
kproto = KPrototypes(n_clusters = 4, init=Huang, verbose=verbose)

NameError: name 'n_clusters' is not defined

In [47]:
clusters = KPrototypes.fit_predict(X, categorical=categ)

TypeError: fit_predict() missing 1 required positional argument: 'X'